# Decision Trees Classification

The representation of the CART model is a binary tree. A node represents a single input variable (X) and a split point on that variable, assuming the variable is numeric. The leaf nodes (also called terminal nodes) of the tree contain an output variable (y) which is used to make a prediction.

Once created, a tree can be navigated with a new row of data following each branch with the splits until a final prediction is made.

Creating a binary decision tree is actually a process of dividing up the input space. A greedy approach is used to divide the space called recursive binary splitting. This is a numerical procedure where all the values are lined up and different split points are tried and tested using a cost function.

The split with the best cost (lowest cost because we minimize cost) is selected. All input variables and all possible split points are evaluated and chosen in a greedy manner based on the cost function.

## 1. Theory on CART - Classification And Regression Trees

### 1.1 Gini Index

Gini index is measure of impurity. It is used as cost function in <b>CART</b> (Classification And Regression Trees) to evaluate splits in dataets. Gini index gives an idea of how good a split is by measuring how mixed the classes are in the two groups created by the split. For example for a two class problem, a perfect separation results in a Gini score of 0, whereas the worst case split that results in 50/50 classes in each group result in a Gini score of 0.5. If a dataset $T$ contains examples from $n$ classes, Gini Index $Gini(T)$ is defined as:

$$Gini(T) = 1 - \sum_{j=1}^{n}p_{j}^{2}$$

If a dataset $T$ is split into two subsets $T_{1}$ and $T_{2}$ with sizes $N_{1}$ and $N_{2}$ respectively, the Gini index of split data contains examples from $n$ classes, te Gini indix $(T)$ is defined as:

$$Gini_{split}(T) = \frac{N_{1}}{N}gini(T_{1}) + \frac{N_{2}}{N}gini(T_{2})$$

### 1.2 Information Gain

Homogeneity of sample can be also calculated using entrophy. If the sample is completely homogeneous the entropy is zero and if the sample is an equally divided it has entropy of one. Entropy $E(T)$ is defined as:

$$E(T) = \sum_{j=1}^{n}-p_{j}log_{2}p_{j}$$

If a dataset $T$ is split into two subsets $T_{1}$ and $T_{2}$ with sizes $N_{1}$ and $N_{2}$ respectively, the Gini index of split data contains examples from $n$ classes, te Gini indix $(T)$ is defined as:

$$E(T, X) = \frac{N_{1}}{N}E(T_{1}) + \frac{N_{2}}{N}E(T_{2})$$

### 1.3 Avoiding over-fitting in decision trees

Overfitting refers to a model that models the training data too well. In overfitting, a statistical model describes random error or noise instead of the underlying relationship. A model that has been overfitted has poor predictive performance, as it overreacts to minor fluctuations in the training data.

Overfitting is more likely with nonparametric and nonlinear models that have more flexibility when learning a target function. Decision trees are a nonparametric machine learning algorithm that is very flexible and is subject to overfitting training data. Preventing overfitting is pivotal while modeling a decision tree and it can be done in 2 ways:

* <b>Tree pruning</b>
* <b>Setting constraints on tree size</b>

   * Minimum samples for a node split
   * Minimum samples for a terminal node
   * Maximum depth of tree

## 2. Importing and preperation of data

### 2.1 Import libraries

In [1]:
import numpy as np
import pandas as pd

### 2.2 Load dataset

<b>NOTE:</b> The Titanic data set contains information on the passengers of the famous ship that sank on April 15, 1912. Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. The dataset includes among others information on gender, age, cabin of passenger and label informing if the passenger drowned.

In [2]:
# Importing the dataset
dataset_test = pd.read_csv('../../00_Datasets/Titanic_test.csv', header = 0, dtype={'Age': np.float64})
dataset_train = pd.read_csv('../../00_Datasets/Titanic_train.csv', header = 0, dtype={'Age': np.float64})
full_data = [dataset_test, dataset_train]

### 2.3 Summarize the Dataset

In [3]:
dataset_train.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
dataset_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


Right away we see that data set contains columns with categorical data along with informations that aren't really useful in current form like number of cabin. Our job is to preprocess data and extract from those columns essential informations. This part of notebook is based on work of Sina [5].

### 2.4 Data preprocessing

### 2.4.1 Pclass

There is no missing values on this feature and it is already a numerical value. Let's check it's impact on our train set:

In [5]:
print (dataset_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=False).mean())

   Pclass  Survived
0       1  0.629630
1       2  0.472826
2       3  0.242363


### 2.4.2 Sex

In [6]:
print (dataset_train[["Sex", "Survived"]].groupby(['Sex'], as_index=False).mean())

      Sex  Survived
0  female  0.742038
1    male  0.188908


### 2.4.3 SibSp and Parch

Using the number of siblings/spouse and the number of children/parents we can create new feature called Family Size.

In [7]:
for dataset in full_data:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1
print (dataset_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean())

   FamilySize  Survived
0           1  0.303538
1           2  0.552795
2           3  0.578431
3           4  0.724138
4           5  0.200000
5           6  0.136364
6           7  0.333333
7           8  0.000000
8          11  0.000000


It seems that new feature has a good effect on our prediction, but let's go further and categorize people to check whether they are alone in this ship or not.

In [8]:
for dataset in full_data:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1
print (dataset_train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean())

   IsAlone  Survived
0        0  0.505650
1        1  0.303538


### 2.4.4 Embarked

The embarked feature contains information about port of Embarkation, unfortunately it has some missing values. We try to fill those with the most occurred value ( 'S' ).

In [9]:
for dataset in full_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')
print (dataset_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean())

  Embarked  Survived
0        C  0.553571
1        Q  0.389610
2        S  0.339009


### 2.4.5 Fare

Fare feature also has some missing value. We replace those values with the median, then we categorize it into 4 ranges.

In [10]:
for dataset in full_data:
    # Fill NA/NaN values using the specified method
    dataset['Fare'] = dataset['Fare'].fillna(dataset_train['Fare'].median())
    dataset['CategoricalFare'] = pd.qcut(dataset_train['Fare'], 4)
print (dataset_train[['CategoricalFare', 'Survived']].groupby(['CategoricalFare'], as_index=False).mean())

   CategoricalFare  Survived
0   (-0.001, 7.91]  0.197309
1   (7.91, 14.454]  0.303571
2   (14.454, 31.0]  0.454955
3  (31.0, 512.329]  0.581081


### 2.4.6 Age

We have plenty of missing values in this feature. We fill blanks with random numbers between (mean - std) and (mean + std), then we categorize age into 5 range.

In [11]:
for dataset in full_data:
    age_avg 	   = dataset['Age'].mean()
    age_std 	   = dataset['Age'].std()
    age_null_count = dataset['Age'].isnull().sum()
    
    age_null_random_list = np.random.randint(age_avg - age_std, age_avg + age_std, size=age_null_count)
    dataset['Age'][np.isnan(dataset['Age'])] = age_null_random_list
    dataset['Age'] = dataset['Age'].astype(int)
    
    dataset['CategoricalAge'] = pd.cut(dataset_train['Age'], 5)

print (dataset_train[['CategoricalAge', 'Survived']].groupby(['CategoricalAge'], as_index=False).mean())

  CategoricalAge  Survived
0  (-0.08, 16.0]  0.537037
1   (16.0, 32.0]  0.345372
2   (32.0, 48.0]  0.384615
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


C:\Users\AFGHAN92\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


### 2.4.7 Data Cleaning

In [12]:
for dataset in full_data:
    # Mapping Sex
    dataset['Sex'] = dataset['Sex'].map( {'female': 0, 'male': 1} ).astype(int)
    
    # Mapping Embarked
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
    
    # Mapping Fare
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] 						        = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] 							        = 3
    dataset['Fare'] = dataset['Fare'].astype(int)
    
    # Mapping Age
    dataset.loc[ dataset['Age'] <= 16, 'Age'] 					       = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']                           = 4

# Feature Selection
drop_elements = ['PassengerId', 'Name', 'Ticket', 'Cabin', 'SibSp',\
                 'Parch', 'FamilySize']

dataset_train = dataset_train.drop(drop_elements, axis = 1)
dataset_train = dataset_train.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

dataset_test = dataset_test.drop(drop_elements, axis = 1)
dataset_test = dataset_test.drop(['CategoricalAge', 'CategoricalFare'], axis = 1)

print (dataset_train.head(10))

train = dataset_train.values
test  = dataset_test.values

   Survived  Pclass  Sex  Age  Fare  Embarked  IsAlone
0         0       3    1    1     0         0        0
1         1       1    0    2     3         1        0
2         1       3    0    1     1         0        1
3         1       1    0    2     3         0        0
4         0       3    1    2     1         0        1
5         0       3    1    1     1         2        1
6         0       1    1    3     3         0        1
7         0       3    1    0     2         0        0
8         1       3    0    1     1         0        0
9         1       2    0    0     2         1        0


,Survived,Pclass,Sex,Age,Fare,Embarked,IsAlone
0,0,3,1,1,0,0,0
1,1,1,0,2,3,1,0
2,1,3,0,1,1,0,1
3,1,1,0,2,3,0,0
4,0,3,1,2,1,0,1
5,0,3,1,1,1,2,1
6,0,1,1,3,3,0,1
7,0,3,1,0,2,0,0
8,1,3,0,1,1,0,0
9,1,2,0,0,2,1,0


## 5. Bibliography

1. https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/
2. https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/
3. https://en.wikipedia.org/wiki/Overfitting
4. https://www.kaggle.com/arthurtok/introduction-to-ensembling-stacking-in-python/notebook
5. https://www.kaggle.com/sinakhorami/titanic-best-working-classifier